In [1]:

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive 
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
! mkdir logistic_data
!pwd
%cd logistic_data/
%cp /content/emnist-letters-train.csv /content/drive/MyDrive/logistic_data
%cp /content/emnist-letters-test.csv /content/drive/MyDrive/logistic_data
! ls

Mounted at /content/drive
/content/drive/MyDrive
mkdir: cannot create directory ‘logistic_data’: File exists
/content/drive/MyDrive
/content/drive/MyDrive/logistic_data
cp: cannot stat '/content/emnist-letters-train.csv': No such file or directory
cp: cannot stat '/content/emnist-letters-test.csv': No such file or directory
emnist-letters-test.csv  emnist-letters-train.csv


In [3]:
train=pd.read_csv("emnist-letters-train.csv",header=[0])
test=pd.read_csv("emnist-letters-test.csv",header=[0])
#train.to_numpy()
Y_train=train['23']
Y_train=np.vstack(Y_train)
X_train = train.iloc[: , 1:]
X_train.to_numpy()
X_test = test.iloc[: , 1:]
Y_test=test['1']

print(X_train.shape)

print(Y_train.shape)

(5969, 784)
(5969, 1)


In [4]:
X_train, x_val, Y_train, y_val = train_test_split(X_train, Y_train, 
    test_size=0.2, random_state= 8) # splitting train into train and validation 

In [5]:
X_train=np.array(X_train)
X_test=np.array(X_test)
Y_train=np.array(Y_train)
Y_test=np.array(Y_test)
# to numpy

In [6]:
#Normalization and standardization of data
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [7]:
X_test

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ..., nan, nan, nan]])

In [8]:
class DeepNeuralNetwork():
    def __init__(NN, size, epochs=10, l_rate=0.01):
        NN.size = size
        NN.epochs = epochs
        NN.l_rate = l_rate # learning rate

        # we save all parameters in the neural network in this dictionary
        NN.params = NN.initialization()

    def sigmoid(x): #sigmoid function 
      
        return 1/(1 + np.exp(-x))

    def softmax( x,):
        # Numerically stable with large exponentials
        exps = np.exp(x - x.max())
        
        return exps / np.sum(exps, axis=0)

    def initialization(NN):
        # number of nodes in each layer
        input_layer=NN.size[0] 
        hidden_1=NN.size[1]
        hidden_2=NN.size[2]
        output_layer=NN.size[3]

        params = { #Defining parameters 
            'W1':np.random.randn(hidden_1, input_layer) * np.sqrt(1 / hidden_1),
            'W2':np.random.randn(hidden_2, hidden_1) * np.sqrt(1 / hidden_2),
            'W3':np.random.randn(output_layer, hidden_2) * np.sqrt(1 / output_layer)
        }

        return params

    def forward_pass(NN, X_train):  #input to output layer
        params = NN.params

        #input layer activations becomes sample
        params['A0'] = X_train

        #input layer to hidden layer 1
        params['B0'] = np.dot(params["W1"], params['A0'])
        params['A1'] = NN.sigmoid(params['B0'])

        #hidden layer 1 to hidden layer 2
        params['B1'] = np.dot(params["W2"], params['A1'])
        params['A2'] = NN.sigmoid(params['B1'])

        #hidden layer 2 to output layer
        params['B2'] = np.dot(params["W3"], params['A2'])
        params['A3'] = NN.softmax(params['B2'])

        return params['A3']

    def backward_pass(NN, Y_train, output): # output to input layer, updation of weights
       
        params = NN.params
        change_w = {}

        #W3 update
        error = 2 * (output - Y_train) / output.shape[0] * NN.softmax(params['B2'])
        change_w['W3'] = np.outer(error, params['A2'])

        #W2 update
        error = np.dot(params['W3'].T, error) * NN.sigmoid(params['B1'])
        change_w['W2'] = np.outer(error, params['A1'])

        #W1 update
        error = np.dot(params['W2'].T, error) * NN.sigmoid(params['B0'])
        change_w['W1'] = np.outer(error, params['A0'])

        return change_w

    def update_network_parameters(NN, changes_to_w):
    
        for key, value in changes_to_w.items(): #updation of parameters
            NN.params[key] -= NN.l_rate * value

    def compute_accuracy(NN, x_val, y_val): #accuracy funtion for deep neural network model
      
        predictions = [] # array to store hypothised predictions 

        for x, y in zip(x_val, y_val):
            output = NN.forward_pass(x)
            pred = np.argmax(output) # To find highest probability among the various possible outputs
            predictions.append(pred == np.argmax(y)) 
        
        return np.mean(predictions) #mean of all the predicted values

    def train(NN, X_train, Y_train, x_val, y_val):
        
        for iteration in range(NN.epochs):
            for x,y in zip(X_train, Y_train): #mapping variables with same value for ease
                output = NN.forward_pass(x)
                changes_to_w = NN.backward_pass(y, output)
                NN.update_network_parameters(changes_to_w)
            
            accuracy = NN.compute_accuracy(x_val, y_val)
            print( accuracy * 100)

In [9]:
dnn = DeepNeuralNetwork(size=[784, 128, 64, 26]) #converting 784 datapoints into 128, then 64 and finally 26 classes(the alphabet)
dnn.train(X_train, Y_train,x_val,y_val) # applying our model on the train data


TypeError: ignored